In [5]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import cvxpy as cp
from datetime import datetime
from dateutil.relativedelta import relativedelta as rd
import time
import math
from sklearn.metrics import mean_squared_error
from sympy import symbols, Eq, solve
from scipy.optimize import minimize
from scipy.optimize import root

from utils import *

np.set_printoptions(threshold=np.inf)

In [6]:
class Argument(object):
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

argments = {
    'data_path': "../NCSOFT/financial_data",
    'result_path': "../results",
    'start_date': "2018-01-02",
    'end_date': "2018-12-31",
    'index_type': "kospi100",
    'cardinality': 50,
    }

args = Argument(**argments)

In [7]:
# price, return, multifactor data
df_price, df_return, df_multifactor, df_index, start_date, end_date, start_year, end_year = read_data(args)
all_stocks_list = df_return.columns.values

# index type
if args.index_type == "kospi100":
    df_index = df_index['IKS100'].pct_change()
    index_stocks_list = json.load(open(args.data_path + '/stock_list.json'))['코스피100'][args.start_date]
elif args.index_type == "s&p500":
    df_index = df_index['SPI@SPX'].pct_change()
    
# print(df_index)
    
# Get the universe
universe = Universe(args = args, df_price= df_price, df_return=df_return, df_multifactor = df_multifactor, df_index=df_index)
    
# trimmed_universe = universe.get_trimmed_universe_by_stocks(list_of_stock_codes=index_stocks_list)
universe = universe.get_trimmed_universe_by_time(start_datetime=start_date, end_datetime=end_date)

In [8]:
print(universe._get_universe_datetime_info()) # print the universe datetime info


{'start': Timestamp('2018-01-02 00:00:00'), 'end': Timestamp('2018-12-28 00:00:00')}


In [9]:
new_return = np.array(universe.df_return)
new_price = np.array(universe.df_price)
new_multifactor = np.array(universe.df_multifactor)
new_index = np.array(universe.df_index)

num_assets = len(new_return[0])
K = args.cardinality

print(num_assets)

# print(new_return.shape)
# print(weight.shape)
# print(new_index.shape)

2480


In [11]:
def f(w):
    return np.sum((new_return @ w - new_index)**2)
def lagrangian_f(w):
    return f(w) - np.sum(w) - w - num_assets + np.sum(1/(1000000000*w+1))

def lagrangian_gradient(w):
    grad = np.zeros((num_assets,))
    for i in range(num_assets):
        grad[i] = new_return[:,i] + 2 + 1000000000/(1000000000*w[i]+1)**2
    return grad

initial_weight = np.random.rand(num_assets)
initial_weight /= initial_weight.sum()

res = root(lagrangian_gradient,initial_weight)
print(res)


TypeError: 'numpy.float64' object does not support item assignment

# Lagrange for full replication

In [6]:
def objective_function(weight):
    error = new_return @ weight - new_index
    return np.sum(error**2)

def weight_sum_constraint(weight):
    return np.sum(weight) - 1

# initial_weight = np.ones(num_assets) / num_assets
initial_weight = np.random.rand(num_assets)
initial_weight /= initial_weight.sum()
bounds = [(0,1) for _ in range(num_assets)]

constraint = {'type': 'eq', 'fun': weight_sum_constraint}

result = minimize(objective_function, initial_weight, method='SLSQP', bounds=bounds, constraints=constraint)

optimal_weight_full = result.x
print("Optimal weight of full replication:", optimal_weight_full)

Optimal weight of full replication: [0.00000000e+00 9.36963356e-03 4.84237434e-18 9.29887127e-18
 0.00000000e+00 0.00000000e+00 1.32815727e-18 9.23397258e-05
 8.10619453e-18 8.92057019e-18 5.20055611e-18 2.13711975e-18
 0.00000000e+00 1.35336500e-17 1.62194988e-18 5.80323594e-18
 0.00000000e+00 1.02552990e-02 8.68924790e-18 0.00000000e+00
 4.74780685e-18 1.55692020e-17 0.00000000e+00 0.00000000e+00
 1.10483844e-03 3.25253002e-18 1.05681954e-17 0.00000000e+00
 4.93614017e-18 3.30824316e-18 0.00000000e+00 2.36215407e-18
 9.49833500e-19 5.52602506e-02 1.59329410e-03 0.00000000e+00
 2.33362869e-04 0.00000000e+00 5.88160179e-18 0.00000000e+00
 2.78704633e-03 0.00000000e+00 3.96012914e-18 0.00000000e+00
 6.43896106e-18 2.28082176e-17 8.46201794e-18 1.15723659e-18
 9.62979769e-18 0.00000000e+00 6.70532448e-03 0.00000000e+00
 1.17124820e-17 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 8.66046230e-19 3.21919008e-18 2.99074691e-19
 0.00000000e+00 0.00000000e+00 0.00000000e+00 5.3

In [7]:
optimal_error = new_return @ optimal_weight_full - new_index
print(np.sum(optimal_error ** 2))

3.369032040488401e-05


In [8]:
stock2weight_full = {}
for i in range(len(all_stocks_list)):
    stock = all_stocks_list[i]
    stock2weight_full[stock] = optimal_weight_full[i]

portfolio_full = Portfolio(universe)
portfolio_full.update_portfolio(stock2weight_full)

In [9]:
evaluator_full = Evaluator(universe=universe, portfolio=portfolio_full)

print("====================================")
print("evaluating portfolio with full replication")
print_result(evaluator_full)
print("====================================")           

evaluating portfolio with full replication
variance         : 0.0001
AV               : 0.1499
AAR              : -0.1859
CAGR             : -0.1235
cumulative_return: -0.1739
Expected_Shortfall: -0.0222
Information_Ratio: -2.9211
LPM              : 0.0084
sharpe_ratio     : -21.1807
calculate_VaR    : -0.0163
['A000020', 'A000030', 'A000040', 'A000050', 'A000060', 'A000070', 'A000080', 'A000100', 'A000120', 'A000140', 'A000150', 'A000180', 'A000210', 'A000220', 'A000230', 'A000240', 'A000250', 'A000270', 'A000300', 'A000320', 'A000370', 'A000390', 'A000400', 'A000430', 'A000440', 'A000480', 'A000490', 'A000500', 'A000520', 'A000540', 'A000590', 'A000640', 'A000650', 'A000660', 'A000670', 'A000680', 'A000700', 'A000720', 'A000760', 'A000810', 'A000850', 'A000860', 'A000880', 'A000890', 'A000910', 'A000950', 'A000970', 'A000990', 'A001000', 'A001020', 'A001040', 'A001060', 'A001070', 'A001080', 'A001120', 'A001130', 'A001140', 'A001200', 'A001210', 'A001230', 'A001250', 'A001260', 'A001

KeyError: "['A023430'] not in index"

# Lagrange for partial replication

### Ours

In [10]:
def objective_function(weight):
    error = new_return @ weight - new_index
    return np.sum(error**2)

def weight_sum_constraint(weight):
    return np.sum(weight) - 1

def cardinality_constraint(weight):
    eps = 1e-4
    coefficient = 99999999999
    approximated_num_of_stocks = [ 1 - 1 / ( coefficient * w + 1 ) for w in weight ]
    approximated_num_of_stocks = np.sum(approximated_num_of_stocks) 
    print("num of stocks :", approximated_num_of_stocks)
    print("max num of stocks :", K)
    return - approximated_num_of_stocks + K - 1  # (number of stocks) >= (min number of stocks)
    

initial_weight = np.ones(num_assets) / num_assets
# initial_weight = np.random.rand(len(num_assets))
# initial_weight /= initial_weight.sum()
bounds = [(0,1) for _ in range(num_assets)]

constraint = {'type': 'eq', 'fun': weight_sum_constraint,
              'type': 'ineq', 'fun': cardinality_constraint}

result = minimize(objective_function, initial_weight, method='SLSQP', bounds=bounds, constraints=constraint)

optimal_weight_ours = result.x
print("Optimal weight of partial replication with ours:", optimal_weight_ours)

num of stocks : 2479.999938496002
max num of stocks : 50
num of stocks : 2479.999938496002
max num of stocks : 50
num of stocks : 2479.999938496002
max num of stocks : 50
num of stocks : 2479.999938496003
max num of stocks : 50
num of stocks : 2479.999938496003
max num of stocks : 50
num of stocks : 2479.999938496003
max num of stocks : 50
num of stocks : 2479.999938496003
max num of stocks : 50
num of stocks : 2479.999938496003
max num of stocks : 50
num of stocks : 2479.999938496003
max num of stocks : 50
num of stocks : 2479.999938496003
max num of stocks : 50
num of stocks : 2479.999938496003
max num of stocks : 50
num of stocks : 2479.999938496003
max num of stocks : 50
num of stocks : 2479.999938496003
max num of stocks : 50
num of stocks : 2479.999938496003
max num of stocks : 50
num of stocks : 2479.999938496003
max num of stocks : 50
num of stocks : 2479.999938496003
max num of stocks : 50
num of stocks : 2479.999938496003
max num of stocks : 50
num of stocks : 2479.9999384960

In [12]:
optimal_error_ours = new_return @ optimal_weight_ours - new_index
print(np.sum(optimal_error_ours ** 2))

3266.3729010450916


In [13]:
stock2weight_ours = {}
for i in range(len(all_stocks_list)):
    stock = all_stocks_list[i]
    stock2weight_ours[stock] = optimal_weight_ours[i]

print(len(stock2weight_ours))
print(stock2weight_ours['A023430'])

portfolio_ours = Portfolio(universe)
portfolio_ours.update_portfolio(stock2weight_ours)

2480
1.0


In [14]:
print(len(portfolio_ours.investments.keys()))
print(portfolio_ours.investments['A023430'])

2480
1.0


In [15]:
evaluator_ours = Evaluator(universe=universe, portfolio=portfolio_ours)

print("====================================")
print("evaluating portfolio with ours")
print_result(evaluator_ours)
print("====================================")           

evaluating portfolio with ours
variance         : 13.4814
AV               : 58.2865
AAR              : 3.4837
CAGR             : 70551023172285287727686962804173859707842199552.0000
cumulative_return: 71773992553176264580533862851081493213497488766928160200761915998208.0000
Expected_Shortfall: -9.0770
Information_Ratio: 0.0554
LPM              : 2.9907
sharpe_ratio     : 0.9132
calculate_VaR    : -6.0261
['A000020', 'A000030', 'A000040', 'A000050', 'A000060', 'A000070', 'A000080', 'A000100', 'A000120', 'A000140', 'A000150', 'A000180', 'A000210', 'A000220', 'A000230', 'A000240', 'A000250', 'A000270', 'A000300', 'A000320', 'A000370', 'A000390', 'A000400', 'A000430', 'A000440', 'A000480', 'A000490', 'A000500', 'A000520', 'A000540', 'A000590', 'A000640', 'A000650', 'A000660', 'A000670', 'A000680', 'A000700', 'A000720', 'A000760', 'A000810', 'A000850', 'A000860', 'A000880', 'A000890', 'A000910', 'A000950', 'A000970', 'A000990', 'A001000', 'A001020', 'A001040', 'A001060', 'A001070', 'A00108

KeyError: "['A023430'] not in index"

### Forward

In [ ]:
num_assets_forward = num_assets
new_return_forward = new_return
largest_weight = []
largest_stocks = []
all_stocks_list_forward = all_stocks_list

while len(largest_weight) <= K:
    weight_forward = cp.Variable(num_assets_forward)
    error_forward = new_return_forward @ weight_forward - new_index

    objective = cp.Minimize(cp.sum_squares(error_forward))
    constraints = [sum(weight_forward) == 1, weight_forward >= 0]

    problem = cp.Problem(objective, constraints)
    problem.solve(solver='OSQP',verbose=True)
    
    # Find Maximum Weight
    max_idx = np.argmax(weight_forward.value)
    max_weight = weight_forward.value[max_idx]
    max_weight_stock = all_stocks_list_forward[max_idx]
    print("max weight:", max_weight)
    print("max weight stock:", max_weight_stock)
    
    # Remove Maximum Weight
    new_return_forward = pd.drop(columns=max_weight_stock)
    all_stocks_list_forward = np.delete(all_stocks_list_forward, max_idx)
    largest_weight.append(max_weight)
    largest_stocks.append(max_weight_stock)
    
# Finally QP with K stocks
weight_forward = cp.Variable(K)
new_return_forward = new_return[largest_stocks]
error_forward = new_return_forward @ weight_forward - new_index
objective = cp.Minimize(cp.sum_squares(error_forward))
constraints = [sum(weight_forward) == 1, weight_forward >= 0]
problem = cp.Problem(objective, constraints)
problem.solve(solver='OSQP',verbose=True)

optimal_weight_forward = weight_forward.value
print("Optimal weight of forward:", optimal_weight_forward)

In [ ]:
stock2weight_forward = {}
for i in range(len(all_stocks_list_forward)):
    stock = all_stocks_list_forward[i]
    stock2weight_forward[stock] = optimal_weight_forward[i]

portfolio_forward = Portfolio(universe)
portfolio_forward.update_portfolio(stock2weight_forward)

In [ ]:
evaluator_forward = Evaluator(universe=universe, portfolio=portfolio_forward)

print("====================================")
print("evaluating portfolio with forward")
print_result(evaluator_forward)
print("====================================")           

### Backward

In [ ]:
weight_backward = cp.Variable(num_assets)

error_backward = new_return @ weight_backward - new_index

# print(new_return @ weight_backward)
# print()
# print(new_index)

objective = cp.Minimize(cp.sum_squares(error_backward))
constraints = [sum(weight_backward) == 1, weight_backward >= 0]

problem = cp.Problem(objective, constraints)
problem.solve(solver='OSQP',verbose=True)

optimal_weight_backward = weight_backward.value
print("Optimal weight of backward:", optimal_weight_backward)

In [ ]:
stock2weight_backward = {}
for i in range(len(all_stocks_list)):
    stock = all_stocks_list[i]
    stock2weight_backward[stock] = optimal_weight_backward[i]

print(len(stock2weight_backward))
print(stock2weight_backward['A023430'])

portfolio_backward = Portfolio(universe)
portfolio_backward.update_portfolio(stock2weight_backward)

In [ ]:
evaluator_backward = Evaluator(universe=universe, portfolio=portfolio_backward)

print("====================================")
print("evaluating portfolio with backward")
print_result(evaluator_backward)
print("====================================")           